In [1]:
import os 
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
%matplotlib inline 
import warnings 
warnings.filterwarnings('ignore')
#Modeling 
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR 
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from catboost import CatBoostRegressor 
from xgboost import XGBRegressor 
import warnings 



# Model Training 

In [2]:
df = pd.read_csv('data/cleaned_data.csv')

In [3]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score,total_score,average
0,female,group_B,bachelors_degree,standard,none,72,72,74,218,72.67
1,female,group_C,some_college,standard,completed,69,90,88,247,82.33
2,female,group_B,masters_degree,standard,none,90,95,93,278,92.67
3,male,group_A,associates_degree,free_reduced,none,47,57,44,148,49.33
4,male,group_C,some_college,standard,none,76,78,75,229,76.33


In [4]:
print("Categories in 'gender' variable:     ",end=" " )
print(df['gender'].unique())

print("Categories in 'race_ethnicity' variable:  ",end=" ")
print(df['race_ethnicity'].unique())

print("Categories in'parental level of education' variable:",end=" " )
print(df['parental_level_of_education'].unique())

print("Categories in 'lunch' variable:     ",end=" " )
print(df['lunch'].unique())

print("Categories in 'test preparation course' variable:     ",end=" " )
print(df['test_preparation_course'].unique())

Categories in 'gender' variable:      ['female' 'male']
Categories in 'race_ethnicity' variable:   ['group_B' 'group_C' 'group_A' 'group_D' 'group_E']
Categories in'parental level of education' variable: ['bachelors_degree' 'some_college' 'masters_degree' 'associates_degree'
 'high_school' 'some_high_school']
Categories in 'lunch' variable:      ['standard' 'free_reduced']
Categories in 'test preparation course' variable:      ['none' 'completed']


# Separate Features and Target

In [5]:
X = df.drop(columns=['average', 'total_score', 'math_score'], axis= 1)

In [6]:
y = df['math_score']

In [7]:
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group_B,bachelors_degree,standard,none,72,74
1,female,group_C,some_college,standard,completed,90,88
2,female,group_B,masters_degree,standard,none,95,93
3,male,group_A,associates_degree,free_reduced,none,57,44
4,male,group_C,some_college,standard,none,78,75


In [8]:
y.head()

0    72
1    69
2    90
3    47
4    76
Name: math_score, dtype: int64

# Create Column Transformer with 3 types of Transformers 

In [9]:
num_features = X.select_dtypes(exclude='object').columns
cat_features = X.select_dtypes(include='object').columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler() 
oh_transformer = OneHotEncoder() 

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features), 
        ("StandardScaler", numeric_transformer, num_features),
        
    ]
           
)

In [10]:
X = preprocessor.fit_transform(X)

## Train and Test Split 

In [11]:
X_train, X_test, y_train, y_test  = train_test_split(X,y, test_size=0.2, random_state = 37)
X_train.shape, X_test.shape

((800, 19), (200, 19))

# Model Performance Metrics 

In [12]:
def evaluate_model(true, predicted):
    mae = round(mean_absolute_error(true, predicted),5),
    mse = round(mean_squared_error(true, predicted), 5)
    rmse = round(np.sqrt(mean_squared_error(true, predicted)),5)
    r2_square = round(r2_score(true, predicted),5)
    return mae, rmse, r2_square 

# Models 

In [13]:
import pandas as pd

models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}

# Create an empty DataFrame
results_df = pd.DataFrame(columns=["Model", "Train RMSE", "Test RMSE", "Train MAE", "Test MAE", "Train R2", "Test R2"])

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    # Append results to DataFrame
    temp_df = pd.DataFrame({
        "Model": [list(models.keys())[i]],
        "Train RMSE": [model_train_rmse],
        "Test RMSE": [model_test_rmse],
        "Train MAE": [model_train_mae],
        "Test MAE": [model_test_mae],
        "Train R2": [model_train_r2],
        "Test R2": [model_test_r2]
    })
    
    results_df = pd.concat([results_df, temp_df], ignore_index=True)

# Calculate the absolute difference between train and test RMSE
results_df['RMSE Difference'] = abs(results_df['Train RMSE'] - results_df['Test RMSE'])

# Sort the DataFrame based on the RMSE Difference
results_df = results_df.sort_values(by='RMSE Difference', ascending=True)

# Display the sorted DataFrame
results_df 



,Model,Train RMSE,Test RMSE,Train MAE,Test MAE,Train R2,Test R2,RMSE Difference
8,AdaBoost Regressor,5.86376,6.05185,"(4.78831,)","(4.69021,)",0.84694,0.85244,0.18809
2,Ridge,5.38173,5.14395,"(4.28579,)","(4.1297,)",0.87107,0.89339,0.23778
0,Linear Regression,5.40623,5.12662,"(4.29938,)","(4.14062,)",0.86990,0.89411,0.27961
1,Lasso,6.62745,6.28612,"(5.21998,)","(4.98992,)",0.80448,0.84079,0.34133
3,K-Neighbors Regressor,5.93630,6.74805,"(4.74725,)","(5.315,)",0.84313,0.81654,0.81175
7,CatBoosting Regressor,2.98539,5.60938,"(2.32538,)","(4.5098,)",0.96033,0.87323,2.62399
5,Random Forest Regressor,2.29988,6.11858,"(1.80506,)","(4.91913,)",0.97645,0.84917,3.81870
6,XGBRegressor,0.94571,6.39903,"(0.62452,)","(5.18246,)",0.99602,0.83502,5.45332
4,Decision Tree,0.35000,8.42852,"(0.0175,)","(6.67,)",0.99945,0.71378,8.07852


# Examine The Linear Regression Analysis for Explanation

In [14]:
import statsmodels.api as sm

# Assuming 'df' is your original DataFrame and 'target' is the name of the target column
X = df.drop(columns=['math_score', 'average', 'total_score'])
y = df['math_score']

# Convert categorical variables to numerical using one-hot encoding
X = pd.get_dummies(X, drop_first=True)

# Convert boolean variables to integers
X = X.astype(int)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Add a constant to the independent variables (X)
X_train_sm = sm.add_constant(X_train)
X_test_sm = sm.add_constant(X_test)

# Set the column names as columns
X_train_sm.columns = ['const'] + list(X_train.columns)
X_test_sm.columns = ['const'] + list(X_test.columns)

# Create and fit the OLS model
ols_model = sm.OLS(y_train, X_train_sm).fit()

# Print the summary, which includes various statistics and coefficients
print(ols_model.summary())





                            OLS Regression Results                            
Dep. Variable:             math_score   R-squared:                       0.874
Model:                            OLS   Adj. R-squared:                  0.872
Method:                 Least Squares   F-statistic:                     390.1
Date:                Tue, 18 Apr 2023   Prob (F-statistic):               0.00
Time:                        03:05:32   Log-Likelihood:                -2472.8
No. Observations:                 800   AIC:                             4976.
Df Residuals:                     785   BIC:                             5046.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                                   coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------

In [15]:
lr = LinearRegression(fit_intercept=True)
lr = lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
score = r2_score(y_test, y_pred) * 100
print(f"{score:.2f}% of the variation in the math score can be explained by this collection of variables.")


88.04% of the variation in the math score can be explained by this collection of variables.


# Diagnostic Measures


## Residuals 

## Df_Betas 

## Cook's Distance

# Hat Values 